In [1]:
%load_ext cudf.pandas

# Accessing remote data stored on the cloud

In this tutorial, we'll cover the following:
- Finding a cloud hosted Zarr archive of CMIP6 dataset(s)
- Remote data access to a single CMIP6 dataset (sea surface height)
- Calculate future predicted sea level change in 2100 compared to 2015

In [2]:
%%cudf.pandas.profile

import gcsfs
import pandas as pd
import xarray as xr

                                                                                              
                                  Total time elapsed: 3.893 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

## Finding cloud native data

Cloud-native data means data that is structured for efficient querying across the network.
Typically, this means having metadata that describes the entire file in the header of the
file, or having a a separate pointer file (so that there is no need to download everything first).

Quite commonly, you'll see cloud-native datasets stored on these
three object storage providers, though there are many other ones too.

- [Amazon Simple Storage Service (S3)](https://aws.amazon.com/s3)
- [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs)
- [Google Cloud Storage](https://cloud.google.com/storage)

### Getting cloud hosted CMIP6 data

The [Coupled Model Intercomparison Project Phase 6 (CMIP6)](https://en.wikipedia.org/wiki/CMIP6#CMIP_Phase_6)
dataset is a rich archive of modelling experiments carried out to predict the climate change impacts.
The datasets are stored using the [Zarr](https://zarr.dev) format, and we'll go over how to access it.

Sources:
- https://esgf-node.llnl.gov/search/cmip6/
- CMIP6 data hosted on Google Cloud - https://console.cloud.google.com/marketplace/details/noaa-public/cmip6
- Pangeo/ESGF Cloud Data Access tutorial - https://pangeo-data.github.io/pangeo-cmip6-cloud/accessing_data.html

First, let's open a CSV containing the list of CMIP6 datasets available

In [3]:
%%cudf.pandas.profile

df = pd.read_csv("https://cmip6.storage.googleapis.com/pangeo-cmip6.csv")
print(f"Number of rows: {len(df)}")
df.head()

Number of rows: 514818


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


                                                                                                        
                                       Total time elapsed: 3.235 seconds                                
                                     5 GPU function calls in 3.153 seconds                              
                                     0 CPU function calls in 0.000 seconds                              
                                                                                                        
                                                     Stats                                              
                                                                                                        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function           ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ read_csv           │ 1          │ 3.105       │ 3.105       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__len__  │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.head     │ 1          │ 0.009       │ 0.009       │ 0          │ 0.000       │ 0.000       │
│ DataFrame          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__repr__ │ 1          │ 0.039       │ 0.039       │ 0          │ 0.000       │ 0.000       │
└────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Over 5 million rows! Let's filter it down to the variable and experiment
we're interested in, e.g. sea surface height.

For the `variable_id`, you can look it up given some keyword at
https://docs.google.com/spreadsheets/d/1UUtoz6Ofyjlpx5LdqhKcwHFz2SGoTQV2_yekHyMfL9Y

For the `experiment_id`, download the spreadsheet from
https://github.com/ES-DOC/esdoc-docs/blob/master/cmip6/experiments/spreadsheet/experiments.xlsx,
go to the 'experiment' tab, and find the one you're interested in.

Another good place to find the right model runs is https://esgf-node.llnl.gov/search/cmip6
(once you get your head around the acronyms and short names).

Below, we'll filter to CMIP6 experiments matching:
- Sea Surface Height Above Geoid [m] (variable_id: `zos`)
- Shared Socioeconomic Pathway 5 (experiment_id: `ssp585`)

In [4]:
%%cudf.pandas.profile

df_zos = df.query("variable_id == 'zos' & experiment_id == 'ssp585'")
df_zos

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
12081,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp585,r1i1p1f1,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...,NaN,20180701
12082,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp585,r1i1p1f1,Omon,zos,gr,gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...,NaN,20180701
45176,ScenarioMIP,AWI,AWI-CM-1-1-MR,ssp585,r1i1p1f1,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR...,NaN,20181218
54265,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp585,r1i1p1f2,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,NaN,20190219
69190,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp585,r4i1p1f2,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,NaN,20190410
...,...,...,...,...,...,...,...,...,...,...,...
497517,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r2i1p1f2,Omon,zos,gr1,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/...,NaN,20201222
497957,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r10i1p1f2,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/...,NaN,20201222
497958,ScenarioMIP,MIROC,MIROC-ES2L,ssp585,r10i1p1f2,Omon,zos,gr1,gs://cmip6/CMIP6/ScenarioMIP/MIROC/MIROC-ES2L/...,NaN,20201222
502853,ScenarioMIP,EC-Earth-Consortium,EC-Earth3-CC,ssp585,r1i1p1f1,Omon,zos,gn,gs://cmip6/CMIP6/ScenarioMIP/EC-Earth-Consorti...,NaN,20210113


                                                                                                        
                                       Total time elapsed: 0.650 seconds                                
                                     2 GPU function calls in 0.122 seconds                              
                                     1 CPU function calls in 0.427 seconds                              
                                                                                                        
                                                     Stats                                              
                                                                                                        
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function           ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.query    │ 0          │ 0.000       │ 0.000       │ 1          │ 0.427       │ 0.427       │
│ DataFrame          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.__repr__ │ 1          │ 0.122       │ 0.122       │ 0          │ 0.000       │ 0.000       │
└────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame.query

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=119560;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

There's 272 modelled scenarios for SSP5.
Let's just get the URL to the first one in the list for now.

In [5]:
%%cudf.pandas.profile

print(df_zos.zstore.iloc[0])

gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/zos/gn/v20180701/


                                                                                                                   
                                             Total time elapsed: 0.052 seconds                                     
                                           1 GPU function calls in 0.004 seconds                                   
                                           0 CPU function calls in 0.000 seconds                                   
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                      ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ _SeriesIlocIndexer.__getitem… │ 1          │ 0.004       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└───────────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

## Reading from the remote Zarr storage

In many cases, you'll need to first connect to the cloud provider.
The CMIP6 dataset allows anonymous access, but for some cases,
you may need to authenticate.

In [6]:
%%cudf.pandas.profile

fs = gcsfs.GCSFileSystem(token="anon")

                                                                                              
                                  Total time elapsed: 0.031 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Next, we'll need a mapping to the Google Storage object.
This can be done using `fs.get_mapper`.

A more generic way (for other cloud providers) is to use
[`fsspec.get_mapper`](https://filesystem-spec.readthedocs.io/en/latest/api.html#fsspec.get_mapper) instead.

In [7]:
%%cudf.pandas.profile

store = fs.get_mapper(
    "gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/zos/gn/v20180701/"
)

                                                                                              
                                  Total time elapsed: 0.029 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

With that, we can open the Zarr store like so.

In [8]:
%%cudf.pandas.profile

ds = xr.open_zarr(store=store, consolidated=True)
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, y: 576, x: 720, vertex: 4, time: 1032)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(576, 720), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(576, 720, 4), meta=np.ndarray>
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
  * x          (x) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * y          (y) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Dimensions without coordinates: vertex
Data variables:
    zos        (time, y, x) float32 dask.array<chunksize=(61, 576, 720), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                <null ref>
    ...                     ...
    tracking_id:            hdl:21.14100/328fd441-31bb-474f-8fd1-7e3420b72524...
    variable_id:            zos
    variant_info:           N/A
    variant_label:          r1i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/328fd441-31bb-474f-8fd1-7e3420b72524...
    version_id:             v20180701

                                                                                                              
                                          Total time elapsed: 4.298 seconds                                   
                                       300 GPU function calls in 0.134 seconds                                
                                       121 CPU function calls in 0.198 seconds                                
                                                                                                              
                                                        Stats                                                 
                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                 ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ isna                     │ 0          │ 0.000       │ 0.000       │ 20         │ 0.013       │ 0.001       │
│ Index                    │ 3          │ 0.009       │ 0.003       │ 1          │ 0.056       │ 0.056       │
│ Float64Index.__len__     │ 264        │ 0.084       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.copy        │ 6          │ 0.003       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__setattr__ │ 6          │ 0.003       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Index.__len__            │ 0          │ 0.000       │ 0.000       │ 89         │ 0.113       │ 0.001       │
│ Index.copy               │ 0          │ 0.000       │ 0.000       │ 2          │ 0.003       │ 0.001       │
│ Index.__setattr__        │ 0          │ 0.000       │ 0.000       │ 2          │ 0.003       │ 0.001       │
│ Float64Index.__getitem__ │ 18         │ 0.015       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Index.__getitem__        │ 0          │ 0.000       │ 0.000       │ 6          │ 0.008       │ 0.001       │
│ Float64Index.__repr__    │ 3          │ 0.020       │ 0.007       │ 0          │ 0.000       │ 0.000       │
│ Index.__repr__           │ 0          │ 0.000       │ 0.000       │ 1          │ 0.003       │ 0.003       │
└──────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- isna
- Index
- Index.__len__
- Index.copy
- Index.__setattr__
- Index.__getitem__
- Index.__repr__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=178223;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

### Selecting time slices

Let's say we want to calculate sea level change between
2015 and 2100. We can access just the specific time points
needed using [`xr.Dataset.sel`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.sel.html).

In [9]:
%%cudf.pandas.profile

zos_2015jan = ds.zos.sel(time="2015-01-16").squeeze()
zos_2100dec = ds.zos.sel(time="2100-12-16").squeeze()

KeyError: "not all values found in index 'time'. Try setting the `method` keyword argument (example: method='nearest')."

                                                                                                          
                                        Total time elapsed: 32.760 seconds                                
                                      2 GPU function calls in 0.001 seconds                               
                                      1 CPU function calls in 0.001 seconds                               
                                                                                                          
                                                      Stats                                               
                                                                                                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function             ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index.__len__        │ 0          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ Float64Index.__len__ │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
└──────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- Index.__len__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=116723;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

Sea level change would just be 2100 minus 2015.

In [10]:
%%cudf.pandas.profile

sealevelchange = zos_2100dec - zos_2015jan

NameError: name 'zos_2100dec' is not defined

                                                                                              
                                  Total time elapsed: 0.229 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Note that up to this point, we have not actually downloaded any
(big) data yet from the cloud. This is all working based on
metadata only.

To bring the data from the cloud to your local computer, call `.compute`.
This will take a while depending on your connection speed.

In [11]:
%%cudf.pandas.profile

sealevelchange = sealevelchange.compute()

NameError: name 'sealevelchange' is not defined

                                                                                              
                                  Total time elapsed: 0.244 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

We can do a quick plot to show how Sea Level is predicted to change
between 2015-2100 (from one modelled experiment).

In [12]:
%%cudf.pandas.profile

sealevelchange.plot.imshow()

NameError: name 'sealevelchange' is not defined

                                                                                              
                                  Total time elapsed: 0.240 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Notice the blue parts between -40 and -60 South where sea level has dropped?
That's to do with the Antarctic ice sheet losing mass and resulting in a lower
gravitational pull, resulting in a relative decrease in sea level. Over most
of the Northern Hemisphere though, sea level rise has increased between 2015 and 2100.

That's all! Hopefully this will get you started on accessing more cloud-native datasets!